# 05 Model Interpretation and Reporting

In this section, we interpret the best-performing models, analyze feature contributions, and generate actionable insights using SHAP, partial dependence plots, and concise reporting.


## 5.1 Import Libraries and Reload Data/Models

In [ ]:
import shap
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.inspection import PartialDependenceDisplay
import numpy as np

X_test_sel = pd.read_csv('data/processed/X_test_selected.csv')
y_test = pd.read_csv('data/processed/y_test.csv').squeeze()

# Load the model
import joblib
model = joblib.load('models/gbm_model.pkl')
